In [445]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import nltk
from pprint import pprint
from nltk.parse import stanford


#initialize empty dataframe
results = pd.DataFrame()

#unique url identifiers for each race
link = [50896,50901,50906,50911, 50916, 50921, 50926]

In [453]:
#lites

#Dealing with names, especially camel case names made this a lot more difficult than I was anticipating
#but I persevered, and I'm glad I did. This is going to be fun to analyze.
html = np.empty(len(link))
df_list = [None] * len(link)

for i,nums in enumerate(link):
    url = 'http://www.promotocross.com/mx/event-results/{}/26'.format(nums)
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html)

#This is where things got incredibly tricky. Names seemed easy, but 
#names like R J and McElrath made a 3 hour ordeal be a 10 hour ordeal.
    elements = soup.find_all("span", class_="athlete-name mobile".split())

    name_gen = "\n".join("{}".format(el.get_text()) for el in elements)
    
#regex to make all camel case standardized lower
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name_gen)
    name_glob = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
#I got a big blob of new line seperated text, so I split to form a list
    name = name_glob.splitlines()
    del name[0::2]
    
    
    #elements = soup.find_all("span", class_="field-content".split())
    #name_rank = "\n".join("{} {}".format(el['class'], el.get_text()) for el in elements)
#Pulling in their race 1 and 2 results. I tried this a bunch of different ways, but I had an epihony that
#made this very simple. I had to split a #-# format, and wasn't sure. Regex can do that really easily, unbeknownst to me...
    elements = soup.find_all("div", class_="field-content".split())
    spread = "\n".join("{}".format(el.get_text()) for el in elements)
    spread = re.findall('\d{1,2}\-\d{1,2}', spread)
    position_list = []
    for nums in spread:
        place = nums.split('-')
        position_list.append(place)

#Splitting the races and making two lists
    race1 = []
    race2 = []
    for element1,element2 in position_list: 
        race1.append(element1)
        race2.append(element2)
#Making a data frame our of all of these
    df = pd.DataFrame({'Name':name, 'Race 1':race1, 'Race 2':race2}).reset_index()
    df['Overall'] = df['index'] + 1
    df.drop('index',1)
    df_list[i] = df
    
    
print(df_list)

[    index                 Name Race 1 Race 2  Overall
0       0        zach _osborne      1      1        1
1       1         alex _martin      4      4        2
2       2    aaron _plessinger      3      5        3
3       3    adam _cianciarulo      2      8        4
4       4      austin _forkner     11      2        5
5       5        colt _nichols      5      6        6
6       6        joey _savatgy      7      7        7
7       7         justin _hill      8      9        8
8       8  mitchell _oldenburg      6     12        9
9       9   mitchell _harrison      9     11       10
10     10       jeremy _martin     35      3       11
11     11       sean _cantrell     10     13       12
12     12     shane _mc_elrath     16     10       13
13     13     michael _mosiman     12     14       14
14     14         nick _gaines     13     15       15
15     15     cameron _mc_adoo     15     16       16
16     16       luke _renzland     14     20       17
17     17    lorenzo _locur